In [ ]:
import torch
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [ ]:
train_x = train_df.drop(['ID','전화해지여부'], axis = 1)
train_y = train_df['전화해지여부']

In [ ]:
train_x[train_x[:] == 0].count()

가입일            0
음성사서함이용    10988
주간통화시간         1
주간통화횟수         8
주간통화요금         1
저녁통화시간         0
저녁통화횟수         5
저녁통화요금         7
밤통화시간          0
밤통화횟수          0
밤통화요금          0
상담전화건수      6303
dtype: int64

In [ ]:
train_data = train_df.drop('ID', axis = 1)

In [ ]:
# 시간대 별 묶음음
train_data['total_call_time'] = train_data['주간통화시간'] + train_data['저녁통화시간'] + train_data['밤통화시간']
train_data['total_call_cnt'] = train_data['주간통화횟수'] + train_data['저녁통화횟수'] + train_data['밤통화횟수']
train_data['total_call_fee'] = train_data['주간통화요금'] + train_data['저녁통화요금'] + train_data['밤통화요금']

train_data['total_fee_per_time'] = train_data['total_call_fee'] / train_data['total_call_time']

train_data['day_fee_per_time'] = train_data['주간통화요금'] / train_data['주간통화시간']
train_data['dinner_fee_per_time'] = train_data['저녁통화요금'] / train_data['저녁통화시간']
train_data['night_fee_per_time'] = train_data['밤통화요금'] / train_data['밤통화시간']

train_data['voice_mail_since_sign'] = train_data['음성사서함이용'] / train_data['가입일']
train_data['consult_since_sign'] = train_data['상담전화건수'] / train_data['가입일']

train_data['total_service'] = train_data['음성사서함이용'] + train_data['상담전화건수']
train_data['voice_mail_fee'] = train_data['음성사서함이용'] / train_data['total_call_fee']
train_data['consult_fee'] = train_data['상담전화건수'] / train_data['total_call_fee']
train_data['consult_time'] = train_data['상담전화건수'] / train_data['주간통화시간']
train_data['consult_count'] = train_data['상담전화건수'] / train_data['주간통화횟수']

#
train_data.loc[train_data['가입일'] > 475, '가입일'] = 475
train_data.loc[train_data['음성사서함이용'] > 55, '음성사서함이용'] = 55
train_data = train_data.replace(np.inf,0)

In [ ]:
train_data.describe()

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,...,day_fee_per_time,dinner_fee_per_time,night_fee_per_time,voice_mail_since_sign,consult_since_sign,total_service,voice_mail_fee,consult_fee,consult_time,consult_count
count,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,...,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000
mean,158.427020,10.178709,250.563457,159.175728,41.091447,263.247215,158.474536,25.394260,263.438285,157.047417,...,0.183930,0.097251,0.047064,0.174770,0.033108,14.325894,0.163311,0.031475,0.014361,0.025239
std,112.549887,17.025935,84.403350,101.573057,23.179017,74.064922,102.200293,8.146126,73.778416,101.721601,...,0.188713,0.021319,0.034361,0.896157,0.095500,25.030649,0.382575,0.029062,0.178541,0.053710
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,20.800000,20.000000,...,0.000000,0.000000,0.006792,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,63.000000,0.000000,222.800000,88.000000,24.400000,223.700000,87.000000,22.220000,223.700000,87.000000,...,0.103081,0.086233,0.018052,0.000000,0.006920,2.000000,0.000000,0.015631,0.006112,0.007067
50%,118.000000,1.000000,234.000000,117.500000,30.800000,243.000000,112.000000,23.320000,242.800000,108.000000,...,0.132833,0.097686,0.041727,0.004545,0.009852,3.000000,0.013393,0.028978,0.008529,0.009901
75%,228.000000,22.000000,289.600000,223.000000,45.900000,290.300000,223.000000,28.820000,290.500000,222.000000,...,0.207783,0.107290,0.079529,0.099099,0.035242,24.000000,0.256044,0.038797,0.009773,0.028302
max,475.000000,55.000000,481.900000,489.000000,118.600000,481.600000,489.000000,49.980000,481.800000,490.000000,...,20.000000,0.391630,0.398077,40.500000,5.000000,1112.000000,25.849232,0.336289,20.000000,4.000000


In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularPredictor
train_datas = train_data
predictor = TabularPredictor(label = '전화해지여부').fit(train_datas)

No path specified. Models will be saved in: "AutogluonModels/ag-20230320_195216/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230320_195216/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    30200
Train Data Columns: 26
Label Column: 전화해지여부
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11045.21 MB
	Train Data (Original)

[1000]	valid_set's binary_error: 0.088


	0.9124	 = Validation score   (accuracy)
	6.56s	 = Training   runtime
	0.3s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0788


	0.9232	 = Validation score   (accuracy)
	10.7s	 = Training   runtime
	0.29s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.92	 = Validation score   (accuracy)
	37.55s	 = Training   runtime
	0.3s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9196	 = Validation score   (accuracy)
	36.1s	 = Training   runtime
	0.29s	 = Validation runtime
Fitting model: CatBoost ...
	0.9292	 = Validation score   (accuracy)
	268.38s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9172	 = Validation score   (accuracy)
	6.13s	 = Training   runtime
	0.33s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9172	 = Validation score   (accuracy)
	7.4s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9008	 = Validation score   (accuracy)
	27.09s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoost ...
	0.9292	 = Validation score   (accuracy)
	20.72s	 = Training   runtime
	0.53s

[1000]	valid_set's binary_error: 0.0736


	0.9264	 = Validation score   (accuracy)
	17.08s	 = Training   runtime
	0.49s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.9324	 = Validation score   (accuracy)
	2.48s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 485.63s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230320_195216/")


In [ ]:
test_data = test_df.drop('ID', axis = 1)
# 시간대 별 묶음음
test_data['total_call_time'] = test_data['주간통화시간'] + test_data['저녁통화시간'] + test_data['밤통화시간']
test_data['total_call_cnt'] = test_data['주간통화횟수'] + test_data['저녁통화횟수'] + test_data['밤통화횟수']
test_data['total_call_fee'] = test_data['주간통화요금'] + test_data['저녁통화요금'] + test_data['밤통화요금']

test_data['day_fee_per_time'] = test_data['주간통화요금'] / test_data['주간통화시간']
test_data['dinner_fee_per_time'] = test_data['저녁통화요금'] / test_data['저녁통화시간']
test_data['night_fee_per_time'] = test_data['밤통화요금'] / test_data['밤통화시간']

test_data['voice_mail_since_sign'] = test_data['음성사서함이용'] / test_data['가입일']
test_data['consult_since_sign'] = test_data['상담전화건수'] / test_data['가입일']

test_data['total_service'] = test_data['음성사서함이용'] + test_data['상담전화건수']
test_data['voice_mail_fee'] = test_data['음성사서함이용'] / test_data['total_call_fee']
test_data['consult_fee'] = test_data['상담전화건수'] / test_data['total_call_fee']
test_data['consult_time'] = test_data['상담전화건수'] / test_data['주간통화시간']
test_data['consult_count'] = test_data['상담전화건수'] / test_data['주간통화횟수']

test_data['total_fee_per_time'] = test_data['total_call_fee'] / test_data['total_call_time']

test_data.loc[test_data['가입일'] > 475, '가입일'] = 475
test_data.loc[test_data['음성사서함이용'] > 55, '음성사서함이용'] = 55
test_data = test_data.replace(np.inf,0)

In [ ]:
prediction = predictor.predict(test_data)

In [ ]:
print(prediction)

0        0
1        0
2        0
3        0
4        1
        ..
12938    0
12939    0
12940    0
12941    0
12942    0
Name: 전화해지여부, Length: 12943, dtype: int64


In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission.describe()

,전화해지여부
count,12943.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [ ]:
submission['전화해지여부'] = prediction
submission.to_csv('./submit.csv', index = False)